# Business Understanding

The problem for this project is to determine the severity of car crashes on a certain day based on various weather and road conditions. This has many business applications from healthcare systems (having more emergency medical technicians EMTs on call) to a company based on this concept to warn users to be careful on the road when high severity crashes are more likely. 

For the duration of the project we will the goal of this model is to provide a healthcare system information on the predicted severity of vehicle collisions so that they can appropriately designate EMTs and ER resources to deal with the approriate levels of demand.

This is very valuable for hospitals to be able to understand these trends from financial and philanthropic perspectives. Hospitals are also businesses and to be able to optimize their staff for low severity days allows the hospital to spend less money which can work towards the benefit of other systems within the hospital. From a philanthropic perspective providing more staff on high severity days will reduce the amount of injuries which escalate in severity due to lack of efficient or timely care. 

In a more complicated system this type of prediction could help Ambulances avoid high crash severity areas when delivering patients. 

# DATA

To understand and address the need for prediction of vehicle collision severity based on weather conditions we need to find the appropriate data. The IBM Data Science course has a provided us with shared data to as an example but also recommends we look for other data sources. 

We would like to had data features such as, road conditions, lighting, and weather.

I've collected data from [data.gov](data.gov) for ny vehicle collisions as well as weather data from [NOAA](https://www.ncdc.noaa.gov/). I'll be combining these two data sources to understand the weather conditions on the days of a crash so that we can predicte severity based on that. I'll compare this combined datasource against the IBM datasource to ensure we have enough quality data to create a model with our derived data. If not we'll go with the IBM Shared data.

[Documentation for NOAA data](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf)

The code immediately below will be for data cleaning, dropping irrelevant columns and manipulating the non IBM data into a form that is useful.

In [1]:
import pandas as pd
import numpy as np

### IBM SHARED DATA

In [2]:
shared_data = pd.read_csv("data/Data-Collisions.csv")
shared_data.columns

C:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE',
       'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND',
       'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

In [3]:
shared_data.drop(['INCKEY','COLDETKEY','OBJECTID','REPORTNO','SEVERITYCODE.1','INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC','JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC','PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'INATTENTIONIND', 'UNDERINFL','CROSSWALKKEY', 'HITPARKEDCAR'], axis=1, inplace=True)
shared_data.head()

,SEVERITYCODE,X,Y,STATUS,ADDRTYPE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,WEATHER,ROADCOND,LIGHTCOND
0,2,-122.323148,47.703140,Matched,Intersection,Injury Collision,Angles,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,Overcast,Wet,Daylight
1,1,-122.347294,47.647172,Matched,Block,Property Damage Only Collision,Sideswipe,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,Raining,Wet,Dark - Street Lights On
2,1,-122.334540,47.607871,Matched,Block,Property Damage Only Collision,Parked Car,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,Overcast,Dry,Daylight
3,1,-122.334803,47.604803,Matched,Block,Property Damage Only Collision,Other,3,0,0,3,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,Clear,Dry,Daylight
4,2,-122.306426,47.545739,Matched,Intersection,Injury Collision,Angles,2,0,0,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,Raining,Wet,Daylight


In [109]:
shared_data.SEVERITYCODE.value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

In [77]:
print(shared_data.shape)
shared_data.describe()

(194673, 16)


,SEVERITYCODE,X,Y,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT
count,194673.000000,189339.000000,189339.000000,194673.000000,194673.000000,194673.000000,194673.000000
mean,1.298901,-122.330518,47.619543,2.444427,0.037139,0.028391,1.920780
std,0.457778,0.029976,0.056157,1.345929,0.198150,0.167413,0.631047
min,1.000000,-122.419091,47.495573,0.000000,0.000000,0.000000,0.000000
25%,1.000000,-122.348673,47.575956,2.000000,0.000000,0.000000,2.000000
50%,1.000000,-122.330224,47.615369,2.000000,0.000000,0.000000,2.000000
75%,2.000000,-122.311937,47.663664,3.000000,0.000000,0.000000,2.000000
max,2.000000,-122.238949,47.734142,81.000000,6.000000,2.000000,12.000000


In [93]:
shared_data.INCDATE.value_counts().describe()

count    5985.000000
mean       32.526817
std        10.514918
min         1.000000
25%        26.000000
50%        32.000000
75%        39.000000
max        96.000000
Name: INCDATE, dtype: float64

In [107]:
shared_data.INCDATE.sort_values()

9398      2004/01/01 00:00:00+00
7996      2004/01/01 00:00:00+00
6679      2004/01/01 00:00:00+00
13914     2004/01/01 00:00:00+00
4866      2004/01/01 00:00:00+00
                   ...          
191912    2020/05/17 00:00:00+00
192328    2020/05/18 00:00:00+00
191908    2020/05/18 00:00:00+00
193070    2020/05/19 00:00:00+00
192030    2020/05/20 00:00:00+00
Name: INCDATE, Length: 194673, dtype: object

### NY WEATHER DATA 2017-2018

In [4]:
ny_weather = pd.read_csv("data/ny_weather.csv")
temp = ny_weather.copy()

In [57]:
temp.columns

Index(['STATION', 'NAME', 'DATE', 'AWND', 'PRCP', 'SNOW', 'TAVG', 'TSUN',
       'WESD', 'WESF', 'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08',
       'WT09', 'WT11'],
      dtype='object')

In [30]:
temp.head(2)

,STATION,NAME,DATE,AWND,PRCP,SNOW,TAVG,TSUN,WESD,WESF,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT11
0,US1NYWC0003,"WHITE PLAINS 3.1 NNW, NY US",2017-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US1NYWC0003,"WHITE PLAINS 3.1 NNW, NY US",2017-01-04,NaN,0.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
cols = {
    'AWND': 'WINDSPEED',
    'TSUN': 'SUNSHINE',
    'WT01': 'FOG',
    'WT02': 'HEAVY_FOG',
    'WT03': 'THUNDER',
    'WT04': 'ICE_PELLETS',
    'WT05': 'HAIL',
    'WT06': 'GLAZE',
    'WT07': 'DUST',
    'WT08': 'SMOKE',
    'WT09': 'SNOW_BLOW',
    'WT11': 'HIGH_WIND'
}
temp.rename(columns=cols, inplace=True)
temp.drop(['WESD','WESF'], axis=1, inplace=True)
temp.head(2)

,STATION,NAME,DATE,WINDSPEED,PRCP,SNOW,TAVG,SUNSHINE,FOG,HEAVY_FOG,THUNDER,ICE_PELLETS,HAIL,GLAZE,SMOKE,SNOW_BLOW,HIGH_WIND
0,US1NYWC0003,"WHITE PLAINS 3.1 NNW, NY US",2017-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US1NYWC0003,"WHITE PLAINS 3.1 NNW, NY US",2017-01-04,NaN,0.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
ny_weather=temp.copy()
print(ny_weather.shape)
ny_weather.describe()

(58132, 17)


,WINDSPEED,PRCP,SNOW,TAVG,SUNSHINE,FOG,HEAVY_FOG,THUNDER,ICE_PELLETS,HAIL,GLAZE,SMOKE,SNOW_BLOW,HIGH_WIND
count,5757.000000,57012.000000,27099.00000,2193.000000,2.0,1755.0,161.0,283.0,71.0,4.0,24.0,355.0,15.0,17.0
mean,8.357625,0.163761,0.15774,56.143639,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,4.293797,0.390071,1.07763,17.211169,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.000000,0.000000,0.00000,9.000000,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,5.140000,0.000000,0.00000,42.000000,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50%,7.610000,0.000000,0.00000,56.000000,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75%,10.740000,0.110000,0.00000,72.000000,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
max,29.970000,5.810000,68.00000,91.000000,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [98]:
ny_weather.head(1)


,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,LOCATION,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED
0,11/01/2019,5:56,BROOKLYN,40.698048,-73.95497,"(40.698048, -73.95497)",0.0,0.0,0,0,0,0,0,0


In [105]:
ny_weather['DATE'].value_counts().describe()

count    731.000000
mean      79.523940
std        5.226163
min       57.000000
25%       76.000000
50%       79.000000
75%       83.000000
max       94.000000
Name: DATE, dtype: float64

### NY COLLISIONS 2017-2018

In [7]:
ny_vehicle_collision = pd.read_csv("data/ny-vehicle-collisions.csv")
temp = ny_vehicle_collision.copy()
temp.columns

C:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME',
       'OFF STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
       'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
       'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'],
      dtype='object')

In [8]:
temp.drop(['ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME',
           'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
           'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID', 
           'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 
           'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5', 'ZIP CODE'], axis=1,inplace=True)
temp.head(2)

,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,LOCATION,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED
0,11/01/2019,5:56,BROOKLYN,40.698048,-73.95497,"(40.698048, -73.95497)",0.0,0.0,0,0,0,0,0,0
1,11/22/2019,6:00,BROOKLYN,40.655495,-73.88833,"(40.655495, -73.88833)",0.0,0.0,0,0,0,0,0,0


In [9]:
ny_vehicle_collision = temp.copy()
ny_vehicle_collision['CRASH DATE'] = pd.to_datetime(ny_vehicle_collision['CRASH DATE']) #NEED TO CREATE DATE
ny_vehicle_collision = ny_vehicle_collision[ny_vehicle_collision['CRASH DATE'] >= pd.Timestamp('2017-01-01')]
ny_vehicle_collision = ny_vehicle_collision[ny_vehicle_collision['CRASH DATE'] < pd.Timestamp('2019-01-01')]
print(ny_vehicle_collision.shape)
ny_vehicle_collision.describe()

(462569, 14)


,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED
count,433527.000000,433527.000000,462553.000000,462539.000000,462569.000000,462569.000000,462569.000000,462569.000000,462569.000000,462569.000000
mean,40.688027,-73.879958,0.265044,0.001051,0.048155,0.000538,0.020782,0.000080,0.196046,0.000443
std,1.218832,2.886927,0.658445,0.035089,0.228314,0.024816,0.144026,0.009182,0.621096,0.021853
min,0.000000,-201.237060,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.667522,-73.975400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,40.721450,-73.926190,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,40.771200,-73.864355,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,41.126150,0.000000,31.000000,8.000000,27.000000,6.000000,3.000000,2.000000,31.000000,2.000000


In [89]:
ny_vehicle_collision.groupby(['CRASH DATE', 'BOROUGH']).count()

CRASH TIME  LATITUDE  LONGITUDE  LOCATION  \
CRASH DATE BOROUGH                                                    
2017-01-01 BRONX                  41        41         41        41   
           BROOKLYN              105       102        102       102   
           MANHATTAN              63        60         60        60   
           QUEENS                 93        93         93        93   
           STATEN ISLAND          11        10         10        10   
...                              ...       ...        ...       ...   
2018-12-31 BRONX                  64        62         62        62   
           BROOKLYN              130       123        123       123   
           MANHATTAN              78        77         77        77   
           QUEENS                104       101        101       101   
           STATEN ISLAND          14        14         14        14   

                          NUMBER OF PERSONS INJURED  NUMBER OF PERSONS KILLED  \
CRASH DATE BOROUGH                                                              
2017-01-01 BRONX                                 41                        41   
           BROOKLYN                             105                       105   
           MANHATTAN                             63                        63   
           QUEENS                                93                        93   
           STATEN ISLAND                         11                        11   
...                                             ...                       ...   
2018-12-31 BRONX                                 64                        64   
           BROOKLYN                             130                       130   
           MANHATTAN                             78                        78   
           QUEENS                               104                       104   
           STATEN ISLAND                         14                        14   

                          NUMBER OF PEDESTRIANS INJURED  \
CRASH DATE BOROUGH                                        
2017-01-01 BRONX                                     41   
           BROOKLYN                                 105   
           MANHATTAN                                 63   
           QUEENS                                    93   
           STATEN ISLAND                             11   
...                                                 ...   
2018-12-31 BRONX                                     64   
           BROOKLYN                                 130   
           MANHATTAN                                 78   
           QUEENS                                   104   
           STATEN ISLAND                             14   

                          NUMBER OF PEDESTRIANS KILLED  \
CRASH DATE BOROUGH                                       
2017-01-01 BRONX                                    41   
           BROOKLYN                                105   
           MANHATTAN                                63   
           QUEENS                                   93   
           STATEN ISLAND                            11   
...                                                ...   
2018-12-31 BRONX                                    64   
           BROOKLYN                                130   
           MANHATTAN                                78   
           QUEENS                                  104   
           STATEN ISLAND                            14   

                          NUMBER OF CYCLIST INJURED  NUMBER OF CYCLIST KILLED  \
CRASH DATE BOROUGH                                                              
2017-01-01 BRONX                                 41                        41   
           BROOKLYN                             105                       105   
           MANHATTAN                             63                        63   
           QUEENS                                93                        93   
           STATEN ISLAND               

### REASSESSING

At this stage we can see that the data we have gathered has significantly more crashes, but does that mean we have more relevant data points? Let's examine a little more. The primary attributes for our model are the weather. And we see that we can divide the crashes into the 5 boroughs which means our data is primarily limited by the number of dates and boroughs we have to distinguish different weather patterns. Our current data is limited between 2017-2018 which only gives us 731 days x 5 Boroughs = 3655 effective data points. This number is doubled if we use the time of day to generate daylight and nightime lighting situations. but is still not comparable to the almost 120k usable data points in the IBM shared dataset. 

The benefit of using our own data is that the severity code is not defined so we can add more depth to our classification. IBM shared data is a binary classification problem, but with our gathered data we can also add in severity code 3 which includes fatalities. 

### DATA CHOICE

After considering the factors I think that continuing forward with the data I have obtained from NOAA and data.gov but expanding it to include more years will provide enough significant data points to create a model. 

I believe this data will provide more value to our potential stakeholders in a hospital who will also care about the extra granularity in severity with fatalities being included and not just injuries. 

### DATA PREPARATION

Now that we've finished up the data understanding we'll need to combine our data sources and create features. First we'll need to download a larger weather data set. Then we'll combine them by merging ny_vechicle_collision and ny_weather on the borough/station column and date.

Following that we'll have to create features into data that will be easily processed into the model and also create our own severity levels based on the following:

0 - Property Damage
1 - Personal Injury
2 - Fatality

In [116]:
#combine 3 datasets into one larger one
ny_weather1 = pd.read_csv("data/ny_weather_12_16.csv")
ny_weather2 = pd.read_csv("data/ny_weather_16_17.csv")
ny_weather3 = pd.read_csv("data/ny_weather_17_20.csv")

temp = pd.concat([ny_weather1, ny_weather2, ny_weather3], ignore_index=True)



In [117]:
cols = {
    'AWND': 'WINDSPEED',
    'PSUN': 'PSUNSHINE',
    'WT01': 'FOG',
    'WT02': 'HEAVY_FOG',
    'WT03': 'THUNDER',
    'WT04': 'ICE_PELLETS',
    'WT05': 'HAIL',
    'WT06': 'GLAZE',
    'WT07': 'DUST',
    'WT08': 'SMOKE',
    'WT09': 'SNOW_BLOW',
    'WT11': 'HIGH_WIND'
}
#temp.rename(columns=cols, inplace=True)
temp.drop(['ELEVATION','WT10'],axis=1, inplace=True)
ny_weather=temp.copy()
ny_weather.head(2)

,STATION,NAME,LATITUDE,LONGITUDE,DATE,AWND,PRCP,PSUN,SNOW,TAVG,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT11
0,US1NYWC0003,"WHITE PLAINS 3.1 NNW, NY US",41.0639,-73.7722,2013-09-15,NaN,0.00,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US1NYWC0003,"WHITE PLAINS 3.1 NNW, NY US",41.0639,-73.7722,2013-09-16,NaN,0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
#Setup ny_collisions for full dataset
ny_vehicle_collision = pd.read_csv("data/ny-vehicle-collisions.csv")
temp = ny_vehicle_collision.copy()
temp.drop(['ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME',
           'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
           'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID', 
           'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 
           'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5', 'ZIP CODE'], axis=1,inplace=True)
ny_collisions = temp.copy()


C:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [56]:
ny_collisions.head(2)

,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,LOCATION,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED
0,11/01/2019,5:56,BROOKLYN,40.698048,-73.95497,"(40.698048, -73.95497)",0.0,0.0,0,0,0,0,0,0
1,11/22/2019,6:00,BROOKLYN,40.655495,-73.88833,"(40.655495, -73.88833)",0.0,0.0,0,0,0,0,0,0


In [118]:
#Need to convert Date and ny_weather.NAME into boroughs for merging

boroughs = {
    "USC00300958": "BROOKLYN",
    "US1NYKN0003": "BROOKLYN",
    "US1NYKN0025": "BROOKLYN",
    "USC00305796": "BROOKLYN", 
    "US1NYQN0029": "QUEENS",
    "US1NYNY0074": "MANHATTAN",
    
}
#Scrapping individual boroughs because not all measurement stations are active throughout the lifetime of the dataset.
#Makes more sense to look at NYC as a whole.


# found borough boundaries in lat/long 
# https://www1.nyc.gov/assets/planning/download/pdf/data-maps/open-data/nybb_metadata.pdf?ver=18c
boroughs_boundary = [[40.49, 40.91],[-73.69, -74.25]] # [lat,long]

# "crop" weather to NYC
ny_weather_by_date = ny_weather.copy()
ny_weather_by_date = ny_weather_by_date[ny_weather_by_date.LATITUDE > boroughs_boundary[0][0]]
ny_weather_by_date = ny_weather_by_date[ny_weather_by_date.LATITUDE < boroughs_boundary[0][1]]
ny_weather_by_date = ny_weather_by_date[ny_weather_by_date.LONGITUDE < boroughs_boundary[1][0]]
ny_weather_by_date = ny_weather_by_date[ny_weather_by_date.LONGITUDE > boroughs_boundary[1][1]]

#convert DATE to a pd.datetime
ny_weather_by_date.DATE = pd.to_datetime(ny_weather_by_date.DATE)

# date grouped dataframe
ny_weather_by_date = ny_weather_by_date.groupby(['DATE']).mean()

In [124]:
#convert collisions 'CRASH DATE' to pd.datetime
ny_collisions['CRASH_DATE'] = pd.to_datetime(ny_collisions['CRASH DATE'])
ny_collisions.drop(['CRASH DATE'], axis=1, inplace=True)

In [131]:
#merge
ny_weather_crashes = pd.merge(ny_collisions, ny_weather_by_date, right_index=True, left_on='CRASH_DATE')

In [135]:
#drop unnecessary columns
ny_weather_crashes.drop(['BOROUGH', 'LATITUDE_x', 'LONGITUDE_x', 'LOCATION', 'LATITUDE_y', 'LONGITUDE_y'], axis=1,inplace=True)

In [137]:
ny_weather_crashes

,CRASH TIME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CRASH_DATE,...,TAVG,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT11
0,5:56,0.0,0.0,0,0,0,0,0,0,2019-11-01,...,55.666667,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17:40,0.0,0.0,0,0,0,0,0,0,2019-11-01,...,55.666667,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,10:50,0.0,0.0,0,0,0,0,0,0,2019-11-01,...,55.666667,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,19:47,0.0,0.0,0,0,0,0,0,0,2019-11-01,...,55.666667,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,17:45,1.0,0.0,0,0,0,0,1,0,2019-11-01,...,55.666667,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1438921,14:00,0.0,0.0,0,0,0,0,0,0,2014-05-12,...,70.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438939,12:50,0.0,0.0,0,0,0,0,0,0,2014-05-12,...,70.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438988,9:00,0.0,0.0,0,0,0,0,0,0,2014-05-12,...,70.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438995,12:35,0.0,0.0,0,0,0,0,0,0,2014-05-12,...,70.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# TODO: feature creation